## 1.0 Load libraries and find paths to assets

In [3]:
import ee
import os
import geemap
import json
import geopandas as gpd
import pprint as pp
from shapely.geometry import shape

ee.Authenticate()
ee.Initialize()

### 1.1 Paths to the GEE assets

In [4]:
project_dir = 'projects/alpod-412314/assets/' 
region = 'TUK_MRD'

region_path = f'{region}_weekly'
#pp.pp(ee.data.listAssets(project_dir))
#pp.pp(ee.data.listAssets(project_dir + 'region_weekly/'))
#pp.pp(ee.data.listAssets(f'{project_dir}Lake_extractions/'))

# Read bufferd lake polygons
region_buffered_lakes = 'TUK_MRD_AND' #!!! TUK_MRD is called TUK_MRD_AND for the lake regions
lakes_path = f'{project_dir}Lake_extractions/{region_buffered_lakes}_extraction'
lake_polygons = ee.FeatureCollection(lakes_path)

# Read weekly mosaics
weekly_mosaics = ee.data.listAssets(f'{project_dir}region_weekly/')
weekly_mosaics = weekly_mosaics['assets']

# Read the ROI boundaries
pp.pp(f"""*** study region boundaries ***
{ee.data.listAssets(project_dir + 'ROIs')}""")
name = 'MRD_TUK_Anderson'
roi_extent = ee.FeatureCollection(f'projects/alpod-412314/assets/ROIs/{name}_roi')
roi_extent = roi_extent.first().geometry() 

print(roi_extent.getInfo())


('*** study region boundaries ***\n'
 "{'assets': [{'type': 'TABLE', 'name': "
 "'projects/alpod-412314/assets/ROIs/AKCP_roi', 'id': "
 "'projects/alpod-412314/assets/ROIs/AKCP_roi', 'updateTime': "
 "'2024-08-15T21:05:05.608722Z'}, {'type': 'TABLE', 'name': "
 "'projects/alpod-412314/assets/ROIs/MRD_TUK_Anderson_roi', 'id': "
 "'projects/alpod-412314/assets/ROIs/MRD_TUK_Anderson_roi', 'updateTime': "
 "'2024-08-15T21:04:58.092718Z'}, {'type': 'TABLE', 'name': "
 "'projects/alpod-412314/assets/ROIs/MRD_roi', 'id': "
 "'projects/alpod-412314/assets/ROIs/MRD_roi', 'updateTime': "
 "'2024-08-15T21:02:37.837931Z'}, {'type': 'TABLE', 'name': "
 "'projects/alpod-412314/assets/ROIs/TUK_roi', 'id': "
 "'projects/alpod-412314/assets/ROIs/TUK_roi', 'updateTime': "
 "'2024-08-15T21:02:57.590868Z'}, {'type': 'TABLE', 'name': "
 "'projects/alpod-412314/assets/ROIs/YKdelta_roi', 'id': "
 "'projects/alpod-412314/assets/ROIs/YKdelta_roi', 'updateTime': "
 "'2024-08-15T21:03:21.623043Z'}, {'type': 'TAB

### 1.2 (optional) export the study region shapefile

In [5]:
# # Set up the export task
# task = ee.batch.Export.table.toDrive(
#     collection=roi_extent,
#     description=f'{name}_roi_shape',
#     folder='ew_roi_shapes', 
#     fileFormat='SHP'
# )

# # Start the export task
# task.start()

# """
# Round-about way to export locally, don't need the Google Drive. 
# """

# export = ee.FeatureCollection(f'{project_dir}ROIs/{name}_roi').first()
# export_info = export.getInfo()
# geometry_json = export_info['geometry']
# geometry = shape(geometry_json)
# roi_gdf = gpd.GeoDataFrame([{'geometry': geometry, 'roi': f'{name}'}], crs="EPSG:4326")
# os.chdir('/Users/jmaze/Documents/projects/altimetry_lakes_v3/')
# roi_gdf.to_file(f'./data/ew_rois/{name}_roi_shape.shp')

## 2.0 Select weekly mosaics for given timeperiods and ROI

### 2.1 Designate the time period

In [6]:
target_years = list(range(2016, 2024))
target_years = [str(year) for year in target_years]

"""
Adjust the time periods here. 
"""
# June weeks !!!
target_weeks = list(range(22, 27))
# August weeks !!!
#target_weeks = list(range(31, 36))
# September weeks !!!
#target_weeks = list(range(35, 41))

target_weeks = [str(week) for week in target_weeks]

max_observations = len(target_years) * len(target_weeks)

### 2.2 Select the images

In [7]:
target_imgs = []
target_imgs_footprints = []


for mosaic in weekly_mosaics:
    img_id = mosaic['id']
    region_match = region.split('_')[-1]
    temp_region = img_id.split('_')[-3]
    temp_region = temp_region.split('/')[-1]
    temp_year = img_id.split('_')[-2]
    temp_week = img_id.split('_')[-1]

    if temp_week in target_weeks and temp_year in target_years and temp_region == region_match:
        #print(img_id)
        image = ee.Image(img_id)
        image_info = image.getInfo()
        #pp.pp(image_info)
        image_coords = image_info['properties']['system:footprint']['coordinates']
        image_polygon_ee = ee.Geometry.Polygon(image_coords)

        target_imgs.append(image)
        target_imgs_footprints.append(image_polygon_ee)

"""
A bit hacky, 
but I combine all the image footprints.
This generates a bounding box for the entire ROI
"""
print(len(target_imgs))

polygon1 = target_imgs_footprints[0]
polygon2 = target_imgs_footprints[1]
combined_footprint = polygon1.union(polygon2)
for i in range(2, len(target_imgs_footprints)):
    p = target_imgs_footprints[i]
    combined_footprint = combined_footprint.union(p)

"""
Write the combined footprint as a bounding box shapefile.
Might be usefull later. 
"""
# combined_footprint_info = combined_ootprint.getInfo()
# geojson_footprint = json.dumps(combined_footprint_info)
# geometry = shape(json.loads(geojson_footprint))
# roi_gdf = gpd.GeoDataFrame([{'geometry': geometry, 'roi': f'{region}'}], crs="EPSG:4326")
# os.chdir('/Users/jmaze/Documents/projects/altimetry_lakes_v3/')
# roi_gdf.to_file(f'./data/ew_rois/{region}_bbox.shp')


39


'\nWrite the combined footprint as a bounding box shapefile.\nMight be usefull later. \n'

## 3.0 Make a classified image (maybe refactor code?)

In [8]:
"""
From the weekly mosaic AND the original buffered lake polygons, produce a new image.
The new image has one band with integers for all conditions.
0 = Never observed, outside of ROI OR not a prior buffered lake
1 = A valid observation of land on a buffered lake polygon
2 = A valid observation of water on a buffered lake polygon
3 = An invalid observatoin where could or ice cover obscured a lake polygon

!!! There's definitely a cleaner way to do this, I might refactor code later.
"""

classified_images = []

for img in target_imgs:

    # Mask of valid observations over buffered lake polygons no clouds or ice
    # !!! Probably not necessary
    obs_mask = img.unmask(0)
    obs_mask = obs_mask.rename('valid_observation')
    obs_mask = obs_mask.clip(combined_footprint)
    #print(obs_mask.getInfo())
    
    # Mask of valid water observations
    img_wtr = ee.Image.constant(0)
    img_wtr = img_wtr.where(img.select('water_occurance_max').eq(1), 2)
    img_wtr = img_wtr.rename('wtr_occurance')
    img_wtr = img_wtr.clip(combined_footprint)
    
    # Mask of valid land observations
    img_land = ee.Image.constant(0)
    img_land = img_land.where(img.select('water_occurance_max').eq(0), 1)
    img_land = img_land.rename('land_occurance')
    img_land = img_land.clip(combined_footprint)
    
    # Mask of prior lake polygons
    lakes_binary = lake_polygons.reduceToImage(
        properties=['n_lakes'],
        reducer=ee.Reducer.anyNonZero()
    ).neq(0)
    lakes_binary = lakes_binary.rename('buffered_lake')
    lakes_binary = lakes_binary.clip(combined_footprint)

    expr = """
    (wtr_observed == 2) ? 2 :
    (land_observed == 1) ? 1 :
    ((buffered_lake == 1) && (land_observed == 0) && (wtr_observed != 2)) ? 3 :
    0
    """
    classified = ee.Image.constant(0)
    classified = lakes_binary.expression(
        expr,
        {
            'wtr_observed': img_wtr.select('wtr_occurance'),
            'land_observed': img_land.select('land_occurance'),
            'buffered_lake': lakes_binary.select('buffered_lake'),
        }
    )
    
    classified = classified.rename('class')
    img_id = img.getInfo()['id']
    classified = classified.set('mosaic_id', img_id)

    classified_images.append(classified)

In [9]:
info = []
for i in classified_images[0:3]:
    inf = i.getInfo()
    info.append(inf)

pp.pp(info)

[{'type': 'Image',
  'bands': [{'id': 'class',
             'data_type': {'type': 'PixelType',
                           'precision': 'int',
                           'min': 0,
                           'max': 3},
             'crs': 'EPSG:4326',
             'crs_transform': [1, 0, 0, 0, 1, 0]}],
  'properties': {'mosaic_id': 'projects/alpod-412314/assets/region_weekly/TUK_MRD_2016_22'}},
 {'type': 'Image',
  'bands': [{'id': 'class',
             'data_type': {'type': 'PixelType',
                           'precision': 'int',
                           'min': 0,
                           'max': 3},
             'crs': 'EPSG:4326',
             'crs_transform': [1, 0, 0, 0, 1, 0]}],
  'properties': {'mosaic_id': 'projects/alpod-412314/assets/region_weekly/TUK_MRD_2016_23'}},
 {'type': 'Image',
  'bands': [{'id': 'class',
             'data_type': {'type': 'PixelType',
                           'precision': 'int',
                           'min': 0,
                           'm

## 4.0 Calculate the proportion of water occurance in valid observations (maybe refactor later?)

In [10]:
"""
Create a collection...
Then, calculate the total valid water observations 
AND invalid (ice/cloud) observations for each pixel
"""
classified_collection = ee.ImageCollection(classified_images)

def binary_masks_from_classified_img(image, class_value):
    return image.updateMask(image.eq(class_value)).multiply(0).add(1)

# Create images for each class
land_images = classified_collection.map(lambda img: binary_masks_from_classified_img(img.select('class'), 1))
wtr_images = classified_collection.map(lambda img: binary_masks_from_classified_img(img.select('class'), 2))
inval_images = classified_collection.map(lambda img: binary_masks_from_classified_img(img.select('class'), 3))

# Sum the images
wtr_sum_img = wtr_images.sum()
#land_sum_img = land_images.sum()
inval_observations_sum_img = inval_images.sum()

"""
Steps for water occurence percentage:

1. Make a constant image for the max possible observations (i.e. all weeks cloud/ice free)
2. Water occurence fraction = [total water observation / (max possible observations - invalid observations)]
"""
# Calc % water for observed images across each pixel
total_obs = len(target_imgs)
print(f'Highest possible total weeks {max_observations}, but observed total weeks {total_obs}')

max_obs_image = ee.Image.constant(total_obs).clip(roi_extent)
wtr_occurance_frac = wtr_sum_img.divide(max_obs_image.subtract(inval_observations_sum_img))

# Example: Print the result
pp.pp(wtr_occurance_frac.getInfo())

Highest possible total weeks 40, but observed total weeks 39
{'type': 'Image',
 'bands': [{'id': 'class',
            'data_type': {'type': 'PixelType', 'precision': 'float'},
            'dimensions': [10, 5],
            'origin': [-138, 66],
            'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'properties': {'system:footprint': {'type': 'MultiPolygon',
                                     'coordinates': [[[[-132.95414856631382,
                                                        68.97842699955754],
                                                       [-132.95416817194518,
                                                        68.97864849722292],
                                                       [-132.9541681826311,
                                                        68.9784329605255],
                                                       [-132.95414856631382,
                                                        68.97842699955754]]],

## 5.0 Visualize and Export

In [11]:
"""
Not sure why changing the data type slows down the code???
"""
wtr_occurance_frac = wtr_occurance_frac.multiply(100).round()
wtr_occurance_frac = wtr_occurance_frac.uint8()

task = ee.batch.Export.image.toDrive(
    image=wtr_occurance_frac,
    description=f'v3_{region}_years{target_years[0]}-{target_years[-1]}_weeks{target_weeks[0]}-{target_weeks[-1]}',
    folder='sentinel2_exports_uint8',
    fileNamePrefix=f'v3_{region}_years{target_years[0]}-{target_years[-1]}_weeks{target_weeks[0]}-{target_weeks[-1]}',
    scale=10,
    crs='EPSG:4326',
    region=roi_extent,
    maxPixels=1e13
)

task.start()

### 5.1 Visualize a random classified image

In [12]:
# conditional_palette = ['grey', 'brown', 'blue', 'pink']
# conditional_viz = {
#     'band': ['class'],
#     'min': 0,
#     'max': 3,
#     'palette': conditional_palette,
# }

# # Grab a random, classified image to inspect
# conditional_test = ee.Image(classified_images[14])
# print(conditional_test.getInfo())

# # Create a map to display the results
# Map = geemap.Map(center=(65, -135), zoom=4)
# Map.add_basemap('SATELLITE')
# Map.addLayer(conditional_test, conditional_viz, 'Conditional Test')

# Map

### 5.2 Visualize the water occurance fraction

In [13]:
# wtr_frac_viz = {
#     'bands': ['class'],
#     'min': 0,
#     'max': 1,
#     'palette': ['brown', 'green', 'blue']
# }


# Map = geemap.Map(center=(65, -135), zoom=4)
# Map.add_basemap('SATELLITE')
# Map.addLayer(wtr_occurance_frac, wtr_frac_viz, 'water frac')
# Map

In [14]:
# band = wtr_occurance_frac.select('class')

# coordinates = [
#     [-146.42929053, 66.34721564],
#     [-145.73500565, 66.32739946],
#     [-145.74722181, 66.15123003],
#     [-146.51124058, 66.15822638]
# ]

# # Create the polygon geometry
# t = ee.Geometry.Polygon([coordinates])

# histogram = band.reduceRegion(
#     reducer=ee.Reducer.fixedHistogram(min=0, max=1, steps=20),
#     geometry=combined_footprint,
#     scale=10,
#     maxPixels=1e13,
# )
# result = histogram.getInfo()

# pp.pp(result)import matplotlib.pyplot as plt

# class_values, frequencies = zip(*result['class'])

# # Create a bar plot
# plt.figure(figsize=(12, 6))
# plt.bar(class_values, frequencies, width=0.03, align='center')
# plt.xticks(class_values, rotation=45)  # Rotate x-ticks for better visibility
# plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.tight_layout()
# plt.show()